### The target's of work
1. Read the data.
2. Calculate the historical returns for 3 months, 6 months, 12 months, 24 months and 36 months for each of the stocks.
3. Define **Gene** : A fraction of the total capital assigned to a stock.
4. Define **Chromosome** : Set of genes i.e. fractions of total capital assigned to each stock. Sum of each chromosome should be equal to 1.
5. Generate Initial **Population** : A set of randomly generated chromosomes.
6. **Fitness** function (Function). Sharpe ratio.
7. Select **Elite Population** (Function): It filters the elite chromosomes which have highest returns, which was calculated in fitness function.

8. **Mutation**: A function that will perform mutation in a chromosome. Randomly we shall choose 2 numbers between 0, 5 and those elements we shall swap.

9. Crossover: **Heuristic crossover** or **Blend Crossover** uses the ﬁtness values of two parent chromosomes to ascertain the direction of the search. It moves from worst parent to best parent.

10. **Next Generation** (Function): A function which does mutation,mating or crossover and builds a new generation of chromosomes.

11. Iterate the process: Iterate the whole process till their is no change in maximum returns or for fixed number of iterations.

In [ ]:
import numpy as np 
import pandas as pd
import os
from functools import reduce
from datetime import datetime
from pandas_datareader import data

#### 1. Read the data.

In [ ]:
# Setting the begining and ending
today = datetime.now()
year_ago = datetime(today.year-3, today.month, today.day)

In [ ]:
ticker_list = ['GOOG', 'AMZN', 'AMD', 'META', 'TSLA', 'TWTR', 'INTC', 'AAPL', 'NVDA', 'MSFT']
dfs = []

for ticker in ticker_list:
    temp = data.DataReader(ticker, 'yahoo', year_ago, today)
    temp['Date'] = temp.index
    temp.index = range(temp.shape[0])
    temp_new = temp[['Date', 'Adj Close']]
    temp_new.columns = ['Date', ticker]
    dfs.append(temp_new)
    
stocks = reduce(lambda left,right: pd.merge(left,right,on='Date'), dfs)
print(stocks.shape)
stocks.head()

#### 2. Calculate the historical returns for 3 months, 6 months, 12 months, 24 months and 36 months for each of the currency.

**Total stock returns**

In [ ]:
def hist_return(months):
    idx = []
    df = pd.DataFrame()
    
    for mon in months:
        temp = (stocks.iloc[0,1:] - stocks.iloc[mon,1:])/(stocks.iloc[mon,1:])
        idx.append(str(mon)+'_mon_return')
        df = pd.concat([df, temp.to_frame().T], ignore_index=True)
    df.index = idx
    return df    

In [ ]:
history_returns = hist_return([3, 6, 12, 24, 36])
history_returns

#### 3. Define Gene (Scalar): A fraction of the total capital assigned to a stock.

In [ ]:
gene = np.random.rand()
gene

In [ ]:
import time


def gen_mc_grid(rows, cols, n, N): 
    np.random.seed(seed=int(time.time())) 
    
    layouts = np.zeros((n, rows * cols), dtype=np.int32)  
    positionX = np.random.randint(0, cols, size=(N * n * 2))
    positionY = np.random.randint(0, rows, size=(N * n * 2))
    ind_rows = 0  
    ind_pos = 0  
    
    while ind_rows < n:
        layouts[ind_rows, positionX[ind_pos] + positionY[ind_pos] * cols] = 1
        if np.sum(layouts[ind_rows, :]) == N:
            ind_rows += 1
        ind_pos += 1
        if ind_pos >= N * n * 2:
            print("Not enough positions")
            break
    return layouts

In [ ]:
def gen_mc_grid_with_NA_loc(rows, cols, n, N,NA_loc):
    np.random.seed(seed=int(time.time())) 

    layouts = np.zeros((n, rows * cols), dtype=np.int32)  
    layouts_NA= np.zeros((n, rows * cols), dtype=np.int32)  

    for i in NA_loc:
        layouts_NA[:,i-1]=2

    positionX = np.random.randint(0, cols, size=(N * n * 2))
    positionY = np.random.randint(0, rows, size=(N * n * 2))

    ind_rows = 0  
    ind_pos = 0  
    N_count=0

    while ind_rows < n:
        cur_state=layouts_NA[ind_rows, positionX[ind_pos] + positionY[ind_pos] * cols]
        if cur_state!=1 and cur_state!=2:
            layouts[ind_rows, positionX[ind_pos] + positionY[ind_pos] * cols]=1
            layouts_NA[ind_rows, positionX[ind_pos] + positionY[ind_pos] * cols] = 1
            N_count+=1
            if np.sum(layouts[ind_rows, :]) == N:
                ind_rows += 1
                N_count=0
        ind_pos += 1
        if ind_pos >= N * n * 2:
            print("Not enough positions")
            break

    return layouts,layouts_NA

In [ ]:
gen_mc_grid(5, 5, 100, 50)
gen_mc_grid_with_NA_loc(5, 5, 100, 50,range(10))

#### 4. Define Chromosome: Set of genes i.e. fractions of total capital assigned to each stock.

In [ ]:
def chromosome(n):
    ''' Generates set of random numbers whose sum is equal to 1
        Input: Number of stocks.
        Output: Array of random numbers'''
    ch = np.random.rand(n)
    return ch/sum(ch)

In [ ]:
child = chromosome(6)
print(child,sum(child))

#### 5. Generate Initial Population.

In [ ]:
n=10 # Number of stocks
pop_size=100 # initial population

population = np.array([chromosome(n) for _ in range(pop_size)])
print(population.shape)
print(population)

#### 6. Fitness function: The Sharpe ratio.

In [ ]:
history_returns[history_returns.columns] = history_returns[history_returns.columns].apply(pd.to_numeric, errors='coerce')
history_returns.info()

*Calculate covariance.*

In [ ]:
cov_hist_return = history_returns.cov()

for i in range(10):
    cov_hist_return.iloc[i][i]=0
    
cov_hist_return

*Calculate mean returns.*

In [ ]:
mean_hist_return=history_returns.mean()
mean_hist_return

*Calculate variation of returns.*

In [ ]:
sd_hist_return=history_returns.std()
sd_hist_return

*Expected returns of portfolio.*

In [ ]:
def mean_portfolio_return(child):
    return np.sum(np.multiply(child,mean_hist_return))

In [ ]:
mean_portfolio_return(population[0])

*Portfolio variance.*

In [ ]:
def var_portfolio_return(child):
    part_1 = np.sum(np.multiply(child,sd_hist_return)**2)
    temp_lst=[]
    for i in range(6):
        for j in range(6):
            temp=cov_hist_return.iloc[i][j] * child[i] * child[j]
            temp_lst.append(temp)
    part_2=np.sum(temp_lst)
    return part_1+part_2

In [ ]:
var_portfolio_return(population[0])

*Risk free factor*

In [ ]:
rf= 0.0697

*Sharpe ratio.*

In [ ]:
def fitness_fuction(child):
    ''' This will return the Sharpe ratio for a particular portfolio.
        Input: A child/chromosome (1D Array)
        Output: Sharpe Ratio value (Scalar)'''
    return (mean_portfolio_return(child) - rf) / np.sqrt(var_portfolio_return(child))

In [ ]:
fitness_fuction(population[7])

#### 7. Select Elite Population.

In [ ]:
def Select_elite_population(population, frac=0.3):
    population = sorted(population,key = lambda x: fitness_fuction(x),reverse=True)
    percentage_elite_idx = int(np.floor(len(population)* frac))
    return population[:percentage_elite_idx]

In [ ]:
print(len(Select_elite_population(population, frac=0.3)))
Select_elite_population(population, frac=0.3)

In [ ]:
[fitness_fuction(x) for x in population][:3]

#### 8. Mutation: A function that will perform mutation in a chromosome.

In [ ]:
def mutation(parent):
    child=parent.copy()
    n=np.random.choice(range(6),2)
    while (n[0]==n[1]):
        n=np.random.choice(range(6),2)
    child[n[0]],child[n[1]]=child[n[1]],child[n[0]]
    return child

In [ ]:
mutation(population[1]),population[1]

#### 9. Crossover: Heuristic crossover or Blend Crossover uses the ﬁtness values of two parent chromosomes to ascertain the direction of the search. It moves from worst parent to best parent.

In [ ]:
def Heuristic_crossover(parent1,parent2):
    '''Off_spring A = Best Parent  + β ∗ ( Best Parent − Worst Parent)
       Off_spring B = Worst Parent - β ∗ ( Best Parent − Worst Parent)
       Where β is a random number between 0 and 1.
       Input: 2 Parents
       Output: 2 Children (1d Array)'''
    ff1=fitness_fuction(parent1)
    ff2=fitness_fuction(parent2)
    diff=parent1 - parent2
    beta=np.random.rand()
    if ff1>ff2:
        child1=parent1 + beta * diff
        child2=parent2 - beta * diff
    else:
        child2=parent1 + beta * diff
        child1=parent2 - beta * diff
    return child1,child2

In [ ]:
def Arithmetic_crossover(parent1,parent2):
    ''' The oﬀsprings are created according to the equation:
        Off spring A = α ∗ Parent1 + (1 −α) ∗ Parent2
        Off spring B = (1 −α) ∗ Parent1 + α ∗ Parent2
        Where α is a random number between 0 and 1.
        Input: 2 Parents
        Output: 2 Children (1d Array)'''
    alpha = np.random.rand()
    child1 = alpha * parent1 + (1-alpha) * parent2
    child2 = (1-alpha) * parent1 + alpha * parent2
    return child1,child2

In [ ]:
for i in population[:30]:
    for j in population[:30]:
        print(Arithmetic_crossover(i,j))

In [ ]:
Arithmetic_crossover(population[2],population[3])

#### 10. Next Generation: A function which does mutation,mating or crossover based on a probability and builds a new generation of chromosomes.

In [ ]:
def next_generation(pop_size,elite,crossover=Heuristic_crossover):
    ''' Generates new population from elite population with mutation probability as 0.4 and crossover as 0.6. 
        Over the final stages, mutation probability is decreased to 0.1.
        Input: Population Size and elite population.
        Output: Next generation population (2D Array).'''
    new_population=[]
    elite_range=range(len(elite))
    
    while len(new_population) < pop_size:
        if len(new_population) > 2*pop_size/3: # In the final stages mutation frequency is decreased.
            mutate_or_crossover = np.random.choice([0, 1], p=[0.9, 0.1])
        else:
            mutate_or_crossover = np.random.choice([0, 1], p=[0.4, 0.6])
        if mutate_or_crossover:
            indx=np.random.choice(elite_range)
            new_population.append(mutation(elite[indx]))
        else:
            p1_idx,p2_idx=np.random.choice(elite_range,2)
            c1,c2=crossover(elite[p1_idx],elite[p2_idx])
            chk=0
            for gene in range(6):
                if c1[gene]<0:
                    chk+=1
                else:
                    chk+=0
            if chk>0:
                p1_idx,p2_idx=np.random.choice(elite_range,2)
                c1,c2=crossover(elite[p1_idx],elite[p2_idx])
            new_population.extend([c1,c2])
    return new_population

#### 11. Iterate the process.

In [ ]:
n=10 # Number of stocks
pop_size=100 # initial population

# Initial population
population = np.array([chromosome(n) for _ in range(pop_size)])

# Get initial elite population
elite = Select_elite_population(population)

iteration=0 
Expected_returns=0
Expected_risk=1

while (Expected_returns < 0.30 and Expected_risk > 0.0005) or iteration <= 40:
    print('Iteration:',iteration)
    population = next_generation(100,elite)
    elite = Select_elite_population(population)
    Expected_returns=mean_portfolio_return(elite[0])
    Expected_risk=var_portfolio_return(elite[0])
    print('Expected returns of {} with risk of {}\n'.format(Expected_returns,Expected_risk))
    iteration+=1

In [ ]:
print('Portfolio of stocks after all the iterations:\n')
[print(history_returns.columns[i],':',elite[0][i]) for i in list(range(6))]

In [ ]:
print('\nExpected returns of {} with risk of {}\n'.format(Expected_returns,Expected_risk))

In [ ]:
fitness_fuction(elite[5])

In [ ]:
n=10 # Number of stocks 
pop_size=100 # initial population

# Initial population
population = np.array([chromosome(n) for _ in range(pop_size)])

# Get initial elite population
elite = Select_elite_population(population)

iteration=0 
Expected_returns=0
Expected_risk=1

while (Expected_returns < 0.30 and Expected_risk > 0.0005) or iteration <= 40:
    print('Iteration:',iteration)
    population = next_generation(100,elite,Arithmetic_crossover)
    elite = Select_elite_population(population)
    Expected_returns=mean_portfolio_return(elite[0])
    Expected_risk=var_portfolio_return(elite[0])
    print('Expected returns of {} with risk of {}\n'.format(Expected_returns,Expected_risk))
    iteration+=1
    if iteration > 15:
        break


print('Portfolio of stocks after all the iterations:\n')
[print(history_returns.columns[i],':',elite[0][i]) for i in list(range(6))]

In [ ]:
print('\nExpected returns of {} with risk of {}\n'.format(Expected_returns,Expected_risk))